In [72]:
import pandas as pd
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import numpy as np

In [73]:
# Import the datasets 
senseData = pd.read_csv("Ltrain.csv", header = None)
senseData.columns = ["Case_ID", "Timestamp", "Type", "Sensor"]
senseData

,Case_ID,Timestamp,Type,Sensor
0,2022-09-02,2022-09-02T08:03:44.275852343,PresenceSensor,door_A
1,2022-09-02,2022-09-02T08:03:49.518493029,FloorSensor,sens3_A
2,2022-09-02,2022-09-02T08:04:19.518493029,FloorSensor,sens3_A
3,2022-09-02,2022-09-02T08:04:49.518493029,FloorSensor,sens3_A
4,2022-09-02,2022-09-02T08:04:55.554026934,FloorSensor,sens4_A
...,...,...,...,...
5814,2022-12-10,2022-12-10T08:37:04.449572377,FloorSensor,sens2_D
5815,2022-12-10,2022-12-10T08:37:34.449572377,FloorSensor,sens2_D
5816,2022-12-10,2022-12-10T08:38:04.449572377,FloorSensor,sens2_D
5817,2022-12-10,2022-12-10T08:38:34.449572377,FloorSensor,sens2_D


In [74]:
newdataset = pd.DataFrame(columns =["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
temp = 0
i = 0

for index, row in senseData.iterrows():
    activity = row["Sensor"][-1]
    new_row = {}
    new_row["Case_ID"] = row["Case_ID"]
    new_row["Timestamp"] = row["Timestamp"]
    new_row["Type"] = row["Type"]
    new_row["Sensor"] = row["Sensor"]
    # First row of the dataset
    if(temp == 0 and i == 0):
        new_row["Label"] = activity
        new_row["Activity_ID"] = i
        temp = activity
    # if the actual row has the same label of the stored, then is the same group
    elif(activity == temp):
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    # if the label mismatch, start a new group
    elif(activity != temp):
        i = i+1
        temp = activity
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    newdataset = newdataset.append(new_row, ignore_index=True)
newdataset

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02T08:03:44.275852343,PresenceSensor,door_A,A,0
1,2022-09-02,2022-09-02T08:03:49.518493029,FloorSensor,sens3_A,A,0
2,2022-09-02,2022-09-02T08:04:19.518493029,FloorSensor,sens3_A,A,0
3,2022-09-02,2022-09-02T08:04:49.518493029,FloorSensor,sens3_A,A,0
4,2022-09-02,2022-09-02T08:04:55.554026934,FloorSensor,sens4_A,A,0
...,...,...,...,...,...,...
5814,2022-12-10,2022-12-10T08:37:04.449572377,FloorSensor,sens2_D,D,399
5815,2022-12-10,2022-12-10T08:37:34.449572377,FloorSensor,sens2_D,D,399
5816,2022-12-10,2022-12-10T08:38:04.449572377,FloorSensor,sens2_D,D,399
5817,2022-12-10,2022-12-10T08:38:34.449572377,FloorSensor,sens2_D,D,399


In [75]:
# Convert the date columns in datetime type
newdataset['Timestamp'] = pd.to_datetime(newdataset['Timestamp'], format="%Y-%m-%dT%H:%M:%S.%f")
newdataset

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02 08:03:44.275852343,PresenceSensor,door_A,A,0
1,2022-09-02,2022-09-02 08:03:49.518493029,FloorSensor,sens3_A,A,0
2,2022-09-02,2022-09-02 08:04:19.518493029,FloorSensor,sens3_A,A,0
3,2022-09-02,2022-09-02 08:04:49.518493029,FloorSensor,sens3_A,A,0
4,2022-09-02,2022-09-02 08:04:55.554026934,FloorSensor,sens4_A,A,0
...,...,...,...,...,...,...
5814,2022-12-10,2022-12-10 08:37:04.449572377,FloorSensor,sens2_D,D,399
5815,2022-12-10,2022-12-10 08:37:34.449572377,FloorSensor,sens2_D,D,399
5816,2022-12-10,2022-12-10 08:38:04.449572377,FloorSensor,sens2_D,D,399
5817,2022-12-10,2022-12-10 08:38:34.449572377,FloorSensor,sens2_D,D,399


In [76]:
out = pd.DataFrame(columns = ["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
i = 0
for idc, case in newdataset.groupby("Case_ID"):
    for id, window in case.groupby(pd.Grouper(freq="30s", key = "Timestamp")):
        if(window.empty):
            pass
        else:
            most_freq_label = window['Label'].value_counts().idxmax()
            dict = {"Case_ID":[],"Timestamp":[],"Type":[],"Sensor":[],"Label":[],"Activity_ID":[]}
            for a, el in window.iterrows():
                dict.get("Case_ID").append(el.get("Case_ID"))
                dict.get("Timestamp").append(el.get("Timestamp"))
                dict.get("Type").append(el.get("Type"))
                dict.get("Sensor").append(el.get("Sensor"))
                dict.get("Label").append(most_freq_label)
                dict.get("Activity_ID").append(i)
            i = i+1
            df2 = pd.DataFrame(dict)
            out = pd.concat([out, df2], ignore_index = True)
            # if(i==20):
            #     break

In [77]:
out

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02 08:03:44.275852343,PresenceSensor,door_A,A,0
1,2022-09-02,2022-09-02 08:03:49.518493029,FloorSensor,sens3_A,A,0
2,2022-09-02,2022-09-02 08:04:19.518493029,FloorSensor,sens3_A,A,1
3,2022-09-02,2022-09-02 08:04:49.518493029,FloorSensor,sens3_A,A,2
4,2022-09-02,2022-09-02 08:04:55.554026934,FloorSensor,sens4_A,A,2
...,...,...,...,...,...,...
5814,2022-12-10,2022-12-10 08:37:04.449572377,FloorSensor,sens2_D,D,4779
5815,2022-12-10,2022-12-10 08:37:34.449572377,FloorSensor,sens2_D,D,4780
5816,2022-12-10,2022-12-10 08:38:04.449572377,FloorSensor,sens2_D,D,4781
5817,2022-12-10,2022-12-10 08:38:34.449572377,FloorSensor,sens2_D,D,4782


In [96]:
# Save the file
out.to_csv("Ltrain_windowed_labeled.csv", index=False)

In [97]:
columns = ["Start Time", "End Time"] + out.Sensor.unique().tolist() + ["Label"]
labeled_senseData = pd.DataFrame(columns=columns)

In [94]:
# Transpose the log and create a row for each group
grouped = out.groupby("Activity_ID")
i = 0
for a, group in grouped:
    group = group.reset_index()
    newrow = {} 
    newrow["Start Time"] = group.iloc[0]["Timestamp"]
    newrow["End Time"] = group.iloc[-1]["Timestamp"]
    newrow["Label"] = group.iloc[0]["Label"]
    for ind, el in group.iterrows():
        newrow[el["Sensor"]] = 1
    labeled_senseData = labeled_senseData.append(newrow, ignore_index = True)
    i +=1
    # if(i==2):
    #     break
# print(labeled_senseData)
labeled_senseData

{'Sart Time': Timestamp('2022-09-02 08:03:44.275852343'), 'End Time': Timestamp('2022-09-02 08:03:49.518493029'), 'Label': 'A', 'door_A': 1, 'sens3_A': 1}
{'Sart Time': Timestamp('2022-09-02 08:04:19.518493029'), 'End Time': Timestamp('2022-09-02 08:04:19.518493029'), 'Label': 'A', 'sens3_A': 1}
{'Sart Time': Timestamp('2022-09-02 08:04:49.518493029'), 'End Time': Timestamp('2022-09-02 08:04:55.554026934'), 'Label': 'A', 'sens3_A': 1, 'sens4_A': 1}
{'Sart Time': Timestamp('2022-09-02 08:05:25.554026934'), 'End Time': Timestamp('2022-09-02 08:05:25.554026934'), 'Label': 'A', 'sens4_A': 1}
{'Sart Time': Timestamp('2022-09-02 08:05:55.554026934'), 'End Time': Timestamp('2022-09-02 08:05:55.554026934'), 'Label': 'A', 'sens4_A': 1}
{'Sart Time': Timestamp('2022-09-02 08:06:25.554026934'), 'End Time': Timestamp('2022-09-02 08:06:25.554026934'), 'Label': 'A', 'sens4_A': 1}
{'Sart Time': Timestamp('2022-09-02 08:06:55.554026934'), 'End Time': Timestamp('2022-09-02 08:06:55.554026934'), 'Label'

,Sart Time,End Time,door_A,sens3_A,sens4_A,sens2_A,sens1_A,door_B,sens4_B,sens3_B,...,sens4_C,sens2_C,sens3_C,sense1_C,door_D,sens1_D,sens3_D,sens2_D,sens4_D,Label
0,2022-09-02 08:03:44.275852343,2022-09-02 08:03:49.518493029,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
1,2022-09-02 08:04:19.518493029,2022-09-02 08:04:19.518493029,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
2,2022-09-02 08:03:44.275852343,2022-09-02 08:03:49.518493029,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
3,2022-09-02 08:04:19.518493029,2022-09-02 08:04:19.518493029,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
4,2022-09-02 08:04:49.518493029,2022-09-02 08:04:55.554026934,NaN,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4781,2022-12-10 08:37:00.414038472,2022-12-10 08:37:04.449572377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,D
4782,2022-12-10 08:37:34.449572377,2022-12-10 08:37:34.449572377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,D
4783,2022-12-10 08:38:04.449572377,2022-12-10 08:38:04.449572377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,D
4784,2022-12-10 08:38:34.449572377,2022-12-10 08:38:34.449572377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,D


In [95]:
# Save the file
labeled_senseData.to_csv("Ltrain_labeled_trans.csv", index=False)